In [2]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_fe.csv')
melb_data.head()
melb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Suburb           13580 non-null  object 
 1   Rooms            13580 non-null  int64  
 2   Type             13580 non-null  object 
 3   Price            13580 non-null  float64
 4   Method           13580 non-null  object 
 5   SellerG          13580 non-null  object 
 6   Date             13580 non-null  object 
 7   Distance         13580 non-null  float64
 8   Postcode         13580 non-null  int64  
 9   Bedroom          13580 non-null  int64  
 10  Bathroom         13580 non-null  int64  
 11  Car              13580 non-null  int64  
 12  Landsize         13580 non-null  float64
 13  BuildingArea     13580 non-null  float64
 14  CouncilArea      12211 non-null  object 
 15  Lattitude        13580 non-null  float64
 16  Longtitude       13580 non-null  float64
 17  Regionname  

In [3]:
melb_df = melb_data.copy()
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
melb_df['Date'].dt.quarter.value_counts()

3    4873
2    4359
4    2329
1    2019
Name: Date, dtype: int64

In [4]:

exclude = ['Date', 'Rooms','Bathroom', 'Bedroom', 'Car']
for col in melb_df.columns:
  if col not in exclude and melb_df[col].nunique() < 150:
    melb_df[col] = melb_df[col].astype('category')
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

In [5]:
melb_df.sort_values(by='Date', ascending=False) # по убыванию даты
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']] # сначала по расстояню а потом по дистанции
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]



,Date,AreaRatio
0,2016-07-26,-0.974922
1,2016-09-24,-0.971831
2,2016-11-27,-0.953608
3,2016-12-11,-0.945946
4,2017-08-04,-0.947368
5,2017-08-04,-0.970874


In [6]:
print(melb_df.sort_values(by = 'AreaRatio',ascending = False, ignore_index=True).loc[1558,'BuildingArea'])
mask1 = melb_df['Type'] == 'townhouse'
mask2 = melb_df['Rooms'] > 2
melb_df[mask1 & mask2].sort_values(
    by = ['Rooms', 'MeanRoomsSquare'],
    ascending = [True, False],
    ignore_index = True
).loc[18,'Price']

126.0


1300000.0

## Группировка данных




In [7]:
display(melb_df.groupby(by='Type').mean())
melb_df.groupby('Type')['Price'].mean()
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)


,Rooms,Price,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,Lattitude,Longtitude,Propertycount,MeanRoomsSquare,AreaRatio,AgeBuilding
Type,,,,,,,,,,,,,,,
house,3.260874,1.242665e+06,10.979479,3104.080643,3.229336,1.613822,1.772674,617.181924,152.162553,-37.803795,144.994700,7259.025505,18.996731,-0.490031,55.669700
townhouse,2.837522,9.337351e+05,9.851346,3100.777379,2.814183,1.809695,1.555655,279.606822,134.649710,-37.815782,144.996489,7094.459605,18.569847,-0.094916,26.690305
unit,1.963871,6.051275e+05,7.607391,3110.797481,1.966523,1.183295,1.128936,477.314219,102.235863,-37.823710,144.996363,8199.280080,21.068242,0.319883,39.703016


Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

In [8]:
melb_df.groupby('MonthSale')['Price'].agg( #несколько агрегирующих методов
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)
# melb_df.groupby('MonthSale')['Price'].agg('describe')


,count,mean,max
MonthSale,,,
8,1850,1.056371e+06,6500000.0
7,1835,9.314698e+05,9000000.0
5,1644,1.097807e+06,8000000.0
6,1469,1.068981e+06,7650000.0
3,1408,1.146762e+06,5600000.0
4,1246,1.050479e+06,5500000.0
9,1188,1.126349e+06,6400000.0
10,854,1.135970e+06,6250000.0
11,750,1.142503e+06,5050000.0


In [9]:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{other, C21, Buxton, RT, Jellis, Ray, Marshall..."
Eastern Victoria,11,"{Eview, Ray, O'Brien, C21, other, Fletchers, M..."
Northern Metropolitan,40,"{Cayzer, other, C21, McDonald, Nick, Raine, Ja..."
Northern Victoria,11,"{Ray, LITTLE, other, McDonald, McGrath, Barry,..."
South-Eastern Metropolitan,25,"{other, C21, Thomson, Buxton, Eview, Jellis, H..."
Southern Metropolitan,38,"{Cayzer, Thomson, other, C21, Buxton, Nick, Ra..."
Western Metropolitan,34,"{other, C21, McDonald, Raine, Jas, Moonee, Swe..."
Western Victoria,6,"{Ray, other, YPA, Raine, hockingstuart, HAR}"


In [10]:
melb_df.groupby(by='Rooms')['Price'].mean().sort_values(ascending=False)

Rooms
7     1.920700e+06
5     1.870260e+06
6     1.849366e+06
8     1.602750e+06
4     1.445282e+06
3     1.076081e+06
10    9.000000e+05
2     7.750812e+05
1     4.338245e+05
Name: Price, dtype: float64

In [11]:
melb_df.groupby('Regionname')['Lattitude'].std().sort_values(ascending=True)

Regionname
Western Victoria              0.011579
Southern Metropolitan         0.043080
Eastern Metropolitan          0.047890
Northern Metropolitan         0.049639
Western Metropolitan          0.051251
South-Eastern Metropolitan    0.073411
Northern Victoria             0.084455
Eastern Victoria              0.147067
Name: Lattitude, dtype: float64

In [12]:
mask = (melb_df['Date']>'2017-05-01') & (melb_df['Date']<='2017-09-01')
melb_df[mask].groupby('SellerG')['Price'].sum().sort_values()

SellerG
LITTLE             2742000.0
Cayzer             4439000.0
Burnham            4550500.0
Moonee             7328000.0
Thomson            8332000.0
Bells              8656000.0
Alexkarbon        10985000.0
McDonald          14637500.0
Rendina           15422276.0
Nick              16890000.0
Douglas           18341000.0
Buckingham        19033000.0
C21               19515000.0
Eview             19791500.0
Collins           20217000.0
Philip            22051800.0
Chisholm          23225000.0
Williams          23297000.0
Love              23365500.0
Purplebricks      23401000.0
O'Brien           23855508.0
HAR               25568000.0
Village           26473000.0
RW                29261000.0
Raine             30687700.0
Stockdale         35409800.0
Sweeney           36882750.0
Gary              39138400.0
Hodges            43231000.0
YPA               46354350.0
Miles             47582000.0
Kay               48569500.0
RT                50498000.0
Brad              55955000.0
Jas   

## Сводные таблицы

In [22]:
display(melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()) # сводная таблица по двум признакам с помощью groupby
melb_df.pivot_table( # метод pivot_table
    values='Price',
    aggfunc='mean',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()


Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,0.0
6,1869508.0,0.0,520000.0
7,1920700.0,0.0,0.0
8,1510286.0,0.0,2250000.0
10,900000.0,0.0,0.0


In [24]:
melb_df.pivot_table(  # многомерная сводная таблица
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                   1244000            780000   
       townhouse                760000                 0   
       unit                     650000                 0   
S      house                   1127000            675000   
       townhouse                828000                 0   
       unit                     645750            492000   
SA     house                    932500            950000   
       townhouse                807500                 0   
       unit                          0                 0   
SP     house                   1050000            672500   
       townhouse                910000                 0   
       unit                     515000            400000   
VB     house                   1100000            712500   
       townhouse                892500                 0   
       unit                     500000                 0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                     900000             500000   
       townhouse                 632500                  0   
       unit                      410000                  0   
S      house                     920000             555000   
       townhouse                 750000                  0   
       unit                      525500                  0   
SA     house                     817500             540000   
       townhouse                 425000                  0   
       unit                      616000                  0   
SP     house                     900000             521000   
       townhouse                 690000                  0   
       unit                      470000                  0   
VB     house                    1050000             690000   
       townhouse                 640000                  0   
       unit                      450000                  0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                          865000                1725000   
       townhouse                     1190000                1055000   
       unit                           525000                 571250   
S      house                          883300                1611000   
       townhouse                      875000                1135000   
       unit                           606000                 655000   
SA     house                          880000                1390000   
       townhouse                           0                1141000   
       unit                                0                 580000   
SP     house                          770000                1521750   
       townhouse                      800000                1162500   
       unit                           601000                 550000   
VB     house                          850000                1800000   
       townhouse                           0                1250000   
       unit                           700000                 500000   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                    870000            630000  
       townhouse                670000                 0  
       unit                     360000                 0  
S      house                    870000            397500  
       townhouse                729000                 0  
       unit                     489000                 0  
SA     house                    772500                 0  
       townhouse                467500                 0  
       unit                     571000                 0  
SP     house                    865000            360000  
       t

In [29]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
display(pivot.head(1))
pivot['mean']['unit'] #последовательное обращение к столбцам 

median                       mean                        
Type                  house townhouse unit       house   townhouse        unit
Regionname                                                                    
Eastern Metropolitan  674.0     233.5  203  717.422847  269.440678  330.444444

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

In [34]:
melb_df.pivot_table(
    values='BuildingArea',
    index='Rooms',
    columns='Type',
    fill_value=0,
    aggfunc='median'
)

Type,house,townhouse,unit
Rooms,,,
1,126.0,88.0,69.5
2,126.0,114.0,110.0
3,126.0,126.0,126.0
4,141.0,159.5,126.0
5,177.0,152.0,0.0
6,126.0,0.0,171.0
7,216.5,0.0,0.0
8,126.0,0.0,126.0
10,126.0,0.0,0.0


In [77]:
# melb_df.pivot_table(
#     values='Price',
#     index='SellerG',
#     columns='Type',
#     fill_value=0,
#     aggfunc='median'
# ).iloc[31]
pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
)
max_unit_price = pivot['unit'].max()
print(pivot[pivot['unit'] == max_unit_price].index[0])
print(pivot.idxmax())

Nick
Type
house           Kay
townhouse    Cayzer
unit           Nick
dtype: object


In [102]:
rate1 = pd.read_csv('data/ratings1.csv')
rate2 = pd.read_csv('data/ratings2.csv')
movies = pd.read_csv('data/movies.csv')
dates = pd.read_csv('data/dates.csv')
dates['date'] = pd.to_datetime(dates['date'])
dates['date'].dt.year.mode()


0    2000
Name: date, dtype: int64

In [105]:
ratings = pd.concat([rate1,rate2],ignore_index=True)
ratings = ratings.drop_duplicates(ignore_index=True) # удаляем дубликаты
ratings_dates = pd.concat([ratings,dates],axis=1)
merge_ratings = rate1.merge(rate2, how='outer') # автоудаление дублей - удобно (для одинаковых столбцов)
print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
display(merge_ratings)
ratings_dates

,userId,movieId,rating,date
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51
...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12


In [107]:
import os

def concat_users_files(path):
    """
    Вам необходимо написать функцию concat_user_files(path), параметром которой является path - путь до директории. 
    Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его. 
    Не забудьте обновить индексы результирующей таблицы после объединения.
    Учтите тот момент, что в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться. 
    """
    user_df = pd.DataFrame()
    files = os.listdir(path)
    for file in sorted(files):
        temp = pd.read_csv(path+file)
        user_df = pd.concat([user_df, temp],ignore_index=True)
    user_df = user_df.drop_duplicates()
    return user_df

if __name__ == '__main__':
    data = concat_users_files('./Root/users/')


['users1.csv', 'users2.csv', 'users3.csv']

In [108]:
joined = ratings_dates.join(
    movies.set_index('movieId'), # метод join (On - ключ в левой таблице по индексам правой(set_index))
    on='movieId',
    how='left'
)
display(joined.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [115]:
merged = ratings_dates.merge( # on - просто название ключевого столбца по которому объединияем 
    movies,                   # если разные названия left_on И right_on
    on='movieId',
    how='left'
)
display(merged.head()) 

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [114]:
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})
purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})
merged = items_df.merge(
    right=purchase_df,   
    on='item_id',
    how = 'inner'
)
display(merged)
merged['total'] = merged['stock_count'] * merged['price']
income = merged['total'].sum()
income

,item_id,vendor,stock_count,purchase_id,price
0,417283,Samsung,54,101,13900
1,849734,LG,33,101,5330
2,132223,Apple,122,101,38200
3,573943,Apple,18,112,49990
4,19475,LG,102,121,9890
5,3294095,Apple,43,145,33000
6,382043,Samsung,77,145,67500
7,302948,Samsung,143,145,34500
8,100132,LG,60,221,11400


19729490

## Practice 

In [128]:
#библиотека для регулярных выражений
from heapq import merge
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
merged['year_release'] = merged['title'].apply(get_year_release)
# merged['year_release'] = pd.to_numeric(merged['year_release'], errors='coerce', downcast='integer')
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   userId        100836 non-null  int64         
 1   movieId       100836 non-null  int64         
 2   rating        100836 non-null  float64       
 3   date          100836 non-null  datetime64[ns]
 4   title         100836 non-null  object        
 5   genres        100836 non-null  object        
 6   year_release  100818 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 6.2+ MB


In [146]:
# merged[merged['year_release']==1999.0].pivot_table(columns='title', values='rating').mean().sort_values()
mask = merged['year_release'] == 1999
merged[mask].groupby('title')['rating'].mean().sort_values()


title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [151]:
merged[merged['year_release']==2010].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [155]:
merged.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [160]:
# merged.pivot_table(
#     index='userId',
#     values='raging'
# )
merged.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by=['count','mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [162]:
mask = merged['year_release'] == 2018
merged[mask].groupby('genres')['rating'].agg(['count', 'mean']).sort_values(by='mean', ascending=False)

,count,mean
genres,,
Adventure|Comedy|Fantasy,1,4.500000
Comedy|Romance,1,4.500000
Comedy|Fantasy|Sci-Fi,1,4.500000
Adventure|Thriller,1,4.500000
Documentary,2,4.000000
Action|Comedy|Crime|Horror,1,4.000000
Action|Crime|Drama|Thriller,1,4.000000
Action|Adventure|Sci-Fi,14,3.928571
Action|Adventure|Children|Sci-Fi,5,3.900000


In [194]:
merged['year_rating']  = merged['date'].dt.year
piv = merged.pivot_table(
    values='rating',
    index='genres',
    columns='year_rating',
    aggfunc='mean'
)
# piv.loc['Action|Adventure|Animation|Children|Comedy|IMAX']
piv.loc[:,2018].sort_values(ascending=False).head(15)
# piv.loc['Comedy']

genres
Animation|Comedy|Drama                       5.0
Crime|Mystery                                5.0
Comedy|Crime|Fantasy                         5.0
Crime|Fantasy|Horror                         5.0
Comedy|Crime|Horror|Thriller                 5.0
Animation|Crime|Drama                        5.0
Drama|Romance|War                            5.0
Action|Comedy|Documentary                    5.0
Animation|Comedy|Fantasy|Musical             5.0
Animation|Comedy|Drama|Fantasy|Sci-Fi        5.0
Comedy|Drama|Fantasy|Mystery|Romance         5.0
Animation|Children|Mystery                   5.0
Drama|Horror|Mystery|Sci-Fi|Thriller         5.0
Adventure|Children|Comedy|Fantasy|Mystery    5.0
Comedy|Fantasy|Horror|Musical|Thriller       5.0
Name: 2018, dtype: float64

In [197]:
orders = pd.read_csv('data/orders.csv', sep=';')
products = pd.read_csv('data/products.csv', sep=';')
display(orders)
display(products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR
5,86,"Носки Простые, муж",45,RUR
6,91,"Носки Честные, муж",50,RUR
7,103,"Носки Подарочные, муж",199,RUR
8,104,"Носки Подарочные, жен",249,RUR
9,124,Носки беговые Camino,999,RUR


In [203]:
orders_products = products.merge(
    right=orders,
    how='right',
    right_on='ID товара',
    left_on= 'Product_ID'
)
orders_products

,Product_ID,Name,Price,CURRENCY,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,103.0,"Носки Подарочные, муж",199.0,RUR,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,86.0,"Носки Простые, муж",45.0,RUR,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,104.0,"Носки Подарочные, жен",249.0,RUR,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,104.0,"Носки Подарочные, жен",249.0,RUR,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,104.0,"Носки Подарочные, жен",249.0,RUR,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,103.0,"Носки Подарочные, муж",199.0,RUR,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,124.0,Носки беговые Camino,999.0,RUR,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,91.0,"Носки Честные, муж",50.0,RUR,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,103.0,"Носки Подарочные, муж",199.0,RUR,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,104.0,"Носки Подарочные, жен",249.0,RUR,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


In [207]:
(orders_products['Price'] * orders_products['Количество']).idxmax()
orders_products.loc[10, 'ID Покупателя']

7

In [208]:
orders_products['Profit'] = orders_products['Price'] * orders_products['Количество'] 
orders_products[orders_products['Оплачен'] == 'Да'].groupby('ID Покупателя')['Profit'].sum().sort_values(ascending=False)


ID Покупателя
7    17096.0
5    13043.0
8     1344.0
1        0.0
Name: Profit, dtype: float64